In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv("/Users/aidencamilleri/Downloads/top5Genes.csv")
df.shape

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'Gene_ID'})
df = df.set_index(keys='Gene_ID')
df.head()

In [ ]:
pca = PCA(n_components=50)
npNew = pca.fit_transform(df)

In [ ]:
dfNew = pd.DataFrame(npNew)

In [ ]:
dfNew.head()

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate='auto', perplexity=50).fit_transform(dfNew)
dfTSNE = pd.DataFrame(X_embedded)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
dfTSNE.columns = ["one", "two"]
dfTSNE.head()

In [ ]:
plt.scatter(dfTSNE["one"], dfTSNE["two"])
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters = 2)
kmeans = kmeans.fit(X_embedded)
npPredict = kmeans.predict(dfTSNE)
dfPredict = pd.DataFrame(npPredict)
dfPredict.columns = ['Prediction']

In [ ]:
dfTSNE.columns = ['one', 'two']
dfPredict = dfTSNE.join(dfPredict)

In [ ]:
plt.scatter(dfPredict["one"], dfPredict["two"], s=15 , c=dfPredict["Prediction"], cmap='cool', alpha=0.3)
plt.show()

In [ ]:
dfPredict.head()

In [ ]:
#df = df.join(dfPredict['Prediction'])
df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
#f.loc[:, df.columns!='Prediction'].head()
X_train, X_test, y_train, y_test = train_test_split(df, dfPredict['Prediction'], test_size=0.33)
clf = RandomForestClassifier(n_estimators=1000, max_depth=4)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
forestPredict = clf.predict(X_test)
print(accuracy_score(y_test, forestPredict, normalize=True))

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(y_test, forestPredict))
sns.set_theme(style='darkgrid')
plt.show()

In [ ]:
print(confusion_matrix(y_test, forestPredict))

In [ ]:
importances = clf.feature_importances_
forest_importances = pd.Series(importances)

In [ ]:
forest_importances.index = (df.loc[:, df.columns!='Prediction']).columns
print(forest_importances)

In [ ]:
forest_importances = forest_importances.loc[lambda x : x > 0.04]
print(forest_importances)

In [ ]:
dfPredict.index = df.index

In [ ]:
dfPredict.head()

In [ ]:
predict1 = dfPredict[dfPredict['Prediction'] == 1]
predict0 = dfPredict[dfPredict['Prediction'] == 0]
predict0.head()

In [ ]:
predict1.head()

In [ ]:
predict1.to_csv('/Users/aidencamilleri/Downloads/predict2.csv')
predict0.to_csv('/Users/aidencamilleri/Downloads/predict3.csv')